In [ ]:
from warnings import filterwarnings

filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
import os
import gc
import random

from tqdm.auto import tqdm
import torch.nn.functional as F
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
def seed_everything(seed=100):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything()

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

IMG_SIZE = 192
BATCH_SIZE = 16

In [ ]:
def encode_labels(masks):
    labelencoder = LabelEncoder()
    masks_reshaped = masks.reshape(-1)
    masks_encoded = labelencoder.fit_transform(masks_reshaped)
    del masks_reshaped, labelencoder
    return masks_encoded.reshape(masks.shape)

In [ ]:
classes = 9


class SegmentationDataset(Dataset):
    def __init__(self, df):
        self.df = df
        images = []
        masks = []
        for idx in range(len(self.df)):
            sample = self.df.iloc[idx]
            images.append(load_img(sample.images, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE)))
            masks.append(load_img(sample.masks, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE)))
        images = np.array(images)
        masks = np.array(masks)
        n_classes = classes  #len(np.unique(masks))
        masks_encoded = encode_labels(masks)
        self.masks_cat_dum = to_categorical(masks_encoded, num_classes=n_classes)
        self.masks_cat = self.masks_cat_dum.copy()
        self.images = np.expand_dims(images, axis=-1) / 255.0
        print(self.masks_cat.shape)
        self.masks_cat[:, :, :, 7], self.masks_cat[:, :, :, 4], self.masks_cat[:, :, :, 1], self.masks_cat[:, :, :,
                                                                                            8], self.masks_cat[:, :, :,
                                                                                                2], self.masks_cat[:, :,
                                                                                                    :,
                                                                                                    5], self.masks_cat[
                                                                                                        :, :, :,
                                                                                                        6], self.masks_cat[
                                                                                                            :, :, :,
                                                                                                            3] = self.masks_cat_dum[
                                                                                                                 :, :,
                                                                                                                 :,
                                                                                                                 1], self.masks_cat_dum[
                                                                                                                     :,
                                                                                                                     :,
                                                                                                                     :,
                                                                                                                     2], self.masks_cat_dum[
                                                                                                                         :,
                                                                                                                         :,
                                                                                                                         :,
                                                                                                                         3], self.masks_cat_dum[
                                                                                                                             :,
                                                                                                                             :,
                                                                                                                             :,
                                                                                                                             4], self.masks_cat_dum[
                                                                                                                                 :,
                                                                                                                                 :,
                                                                                                                                 :,
                                                                                                                                 5], self.masks_cat_dum[
                                                                                                                                     :,
                                                                                                                                     :,
                                                                                                                                     :,
                                                                                                                                     6], self.masks_cat_dum[
                                                                                                                                         :,
                                                                                                                                         :,
                                                                                                                                         :,
                                                                                                                                         7], self.masks_cat_dum[
                                                                                                                                             :,
                                                                                                                                             :,
                                                                                                                                             :,
                                                                                                                                             8]
        del masks_encoded, images, masks, n_classes, self.masks_cat_dum

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image =torch.from_numpy(np.transpose(self.images[idx], (2, 0, 1))).float()
        mask = torch.from_numpy(np.transpose(self.masks_cat[idx], (2, 0, 1))).float()
        return image, mask

In [ ]:
images = os.listdir("Cleaned_DataSet/test")
masks = os.listdir("Cleaned_DataSet/test_labels")
test_df = pd.DataFrame({"images": map(lambda x: os.path.join(r"Cleaned_DataSet/test", x), images),
                        "masks": map(lambda x: os.path.join("Cleaned_DataSet/test_labels", x), masks)})
print(test_df.shape)
del images, masks
test_df.head()

In [ ]:
test_data = SegmentationDataset(test_df)

In [ ]:
testloader_c = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
del test_data, test_df

In [ ]:
for image, mask in testloader_c:
    print(f"Size of one batch of images: {image.shape}")
    print(f"Size of one batch of masks: {mask.shape}")
    break

In [ ]:
images = os.listdir("NR206/test")
masks = os.listdir("NR206/test_labels")
test_df = pd.DataFrame({"images": map(lambda x: os.path.join("NR206/test", x), images),
                        "masks": map(lambda x: os.path.join("NR206/test_labels", x), masks)})
print(test_df.shape)
del images, masks
test_df.head()

In [ ]:
test_data = SegmentationDataset(test_df)

In [ ]:
testloader_n = DataLoader(test_data, batch_size=1, shuffle=False)

print(f"Total number of batches in Val Loader: {len(testloader_n)}")

In [ ]:
del test_data, test_df

In [ ]:
for image, mask in testloader_n:
    print(f"Size of one batch of images: {image.shape}")
    print(f"Size of one batch of masks: {mask.shape}")
    break

In [ ]:
plt.figure(figsize=(50, 5))
for l in range(9):
    plt.subplot(1, 9, l + 1)
    plt.imshow(mask.squeeze().numpy()[l, :, :])
    plt.grid(False)
    plt.title(f"Layer {l + 1}")
plt.show()

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_classes=9, img_channels=1):
        super(UNet, self).__init__()

        # Contraction path
        self.conv1 = self.double_conv(img_channels, 16)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = self.double_conv(16, 32)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = self.double_conv(32, 64)
        self.pool3 = nn.MaxPool2d(2)

        self.conv4 = self.double_conv(64, 128)
        self.pool4 = nn.MaxPool2d(2)

        self.conv5 = self.double_conv(128, 256)

        # Expansive path
        self.up6 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv6 = self.double_conv(256, 128)

        self.up7 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv7 = self.double_conv(128, 64)

        self.up8 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.conv8 = self.double_conv(64, 32)

        self.up9 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.conv9 = self.double_conv(32, 16)

        self.out_conv = nn.Conv2d(16, n_classes, kernel_size=1)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Contraction path
        c1 = self.conv1(x)
        p1 = self.pool1(c1)

        c2 = self.conv2(p1)
        p2 = self.pool2(c2)

        c3 = self.conv3(p2)
        p3 = self.pool3(c3)

        c4 = self.conv4(p3)
        p4 = self.pool4(c4)

        c5 = self.conv5(p4)

        # Expansive path
        u6 = self.up6(c5)
        u6 = torch.cat([u6, c4], dim=1)
        c6 = self.conv6(u6)

        u7 = self.up7(c6)
        u7 = torch.cat([u7, c3], dim=1)
        c7 = self.conv7(u7)

        u8 = self.up8(c7)
        u8 = torch.cat([u8, c2], dim=1)
        c8 = self.conv8(u8)

        u9 = self.up9(c8)
        u9 = torch.cat([u9, c1], dim=1)
        c9 = self.conv9(u9)

        outputs = self.out_conv(c9)
        return outputs

In [ ]:
cm = torch.load('./Clean_DiceLoss_0.3.pt', map_location=DEVICE)

In [ ]:
nm = torch.load('./Noise_DiceLoss_0.3.pt', map_location=DEVICE)

In [ ]:
gc.collect()

In [ ]:
DEVICE

In [ ]:
preds_mean = []
preds_std = []
preds_var = []
trues = []
c_preds = []
cm.train()
for i, (image, mask) in tqdm(enumerate(testloader_c)):
    image = image.to(DEVICE)
    _pred = []
    true = []
    with torch.no_grad():
        for _ in range(50):
            _pred.append(F.sigmoid(cm(image)).cpu().detach().numpy())
            true.append(mask.detach().numpy())
    c_preds.append(_pred)
    trues.append(true)
    preds_mean.append(np.transpose(np.array(_pred).mean(axis=0), (1, 2, 0)))
    preds_std.append(np.transpose(np.array(_pred).std(axis=0), (1, 2, 0)))
    preds_var.append(np.transpose(np.array(_pred).var(axis=0), (1, 2, 0)))
c_preds_mean = torch.Tensor(preds_mean).to(DEVICE)
c_preds_std = torch.Tensor(preds_std).to(DEVICE)
c_preds_var = torch.Tensor(preds_var).to(DEVICE)
c_trues = torch.Tensor(trues).to(DEVICE)
c_preds = torch.Tensor(c_preds).to(DEVICE)
del preds_mean, preds_std, preds_var, trues

In [ ]:
gc.collect()

In [ ]:
preds_mean = []
preds_std = []
preds_var = []
trues = []
n_preds = []
nm.train()
for i, (image, mask) in tqdm(enumerate(testloader_n)):
    image = image.to(DEVICE)
    _pred = []
    true = []
    with torch.no_grad():
        for _ in range(50):
            _pred.append(F.sigmoid(nm(image)).cpu().detach().numpy())
            true.append(mask.detach().numpy())
    n_preds.append(_pred)
    trues.append(true)
    preds_mean.append(np.transpose(np.stack(_pred).mean(axis=0), (1, 2, 0)))
    preds_std.append(np.transpose(np.stack(_pred).std(axis=0), (1, 2, 0)))
    preds_var.append(np.transpose(np.stack(_pred).var(axis=0), (1, 2, 0)))
n_preds_mean = torch.Tensor(preds_mean).to(DEVICE)
n_preds_std = torch.Tensor(preds_std).to(DEVICE)
n_preds_var = torch.Tensor(preds_var).to(DEVICE)
n_trues = torch.Tensor(trues).to(DEVICE)
n_preds = torch.Tensor(n_preds).to(DEVICE)
del preds_mean, preds_std, preds_var, trues

In [ ]:
n_trues, c_trues = n_trues.squeeze(), c_trues.squeeze()

In [ ]:
n_preds.shape, n_trues.shape, c_preds.shape, c_trues.shape

In [ ]:
torch.save(n_preds, "n_preds_gn.pt"), torch.save(c_preds, "c_preds_gn.pt"), torch.save(n_trues, "n_trues_gn.pt"), torch.save(
    c_trues, "c_trues_gn.pt")

# Start Here

In [ ]:
from warnings import filterwarnings

filterwarnings("ignore")
import torch
import numpy as np
import matplotlib.pyplot as plt
import gc

In [ ]:
n_preds=torch.load("./n_preds.pt").squeeze()
n_trues=torch.load("./n_trues.pt").squeeze()
c_preds=torch.load("./c_preds.pt").squeeze()
c_trues=torch.load("./c_trues.pt").squeeze()

In [ ]:
n_preds.shape, n_trues.shape, c_preds.shape, c_trues.shape

In [ ]:
def layer_mul(x, y) -> np.array:
    res = []
    mask_max = 1.0 - torch.max(x, dim=1).values
    for i in range(50):
        layer = []
        for j in range(9):
            layer.append(torch.mul(mask_max[i], y[i, j]).cpu().detach().numpy())
        res.append(layer)
    return np.array(res)

In [ ]:
gc.collect()

In [ ]:
c_layer_uc_var = np.array(
    [np.transpose((layer_mul(x, y)).var(axis=0), (1, 2, 0)) for x, y in zip(c_preds, c_trues)])

c_layer_uc_mean = np.array(
    [np.transpose(layer_mul(x, y).mean(axis=0), (1, 2, 0)) for x, y in zip(c_preds, c_trues)])

c_layer_prob_var = np.array([np.transpose(x.var(axis=0).cpu().detach().numpy(), (1, 2, 0)) for x in c_preds])

c_layer_prob_mean = np.array([np.transpose(x.mean(axis=0).cpu().detach().numpy(), (1, 2, 0)) for x in c_preds])

c_layer_uc_var.shape, c_layer_uc_mean.shape, c_layer_prob_var.shape, c_layer_prob_mean.shape

In [ ]:
n_layer_uc_var = np.array(
    [np.transpose(layer_mul(x, y).var(axis=0), (1, 2, 0)) for x, y in zip(n_preds, n_trues)])

n_layer_uc_mean = np.array(
    [np.transpose(layer_mul(x, y).mean(axis=0), (1, 2, 0)) for x, y in zip(n_preds, n_trues)])

n_layer_prob_var = np.array([np.transpose(x.var(axis=0).cpu().detach().numpy(), (1, 2, 0)) for x in n_preds])

n_layer_prob_mean = np.array([np.transpose(x.mean(axis=0).cpu().detach().numpy(), (1, 2, 0)) for x in n_preds])

n_layer_uc_var.shape, n_layer_uc_mean.shape, n_layer_prob_var.shape, n_layer_prob_mean.shape

In [ ]:
def pcolor_plot(max_val):
    # struct=[(0,191),(60,85),(60,90),(70,95),(75,90),(75,100),(75,100),(80,100),(85,105)]
    struct=[(0,191),(0,191),(0,191),(0,191),(0,191),(0,191),(0,191),(0,191),(0,191)]
    for i, (
            c_img_mean, c_img_var, n_img_mean, n_img_var, c_img_p_mean, n_img_p_mean, c_img_p_var,
            n_img_p_var) in enumerate(
        zip(c_layer_uc_mean, c_layer_uc_var, n_layer_uc_mean, n_layer_uc_var, c_layer_prob_mean, n_layer_prob_mean,
            c_layer_prob_var, n_layer_prob_var)):
        if i==0:
            plt.figure(figsize=(50, 5))
            for l in range(9):
                img=c_img_mean[struct[l][0]:struct[l][1],:, l]
                plt.subplot(1, 9, l + 1)
                plt.imshow(img, vmin=0, vmax=0.05, cmap="jet")
                if l%3==0:
                    plt.colorbar()
                plt.grid(False)
                plt.title(f"Denoised Mean of Uncertainty Layer {l + 1}")
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                img=n_img_mean[struct[l][0]:struct[l][1],:, l]
                plt.subplot(1, 9, l + 1)
                plt.imshow(img, vmin=0, vmax=0.05, cmap="jet")
                if l%3==0:
                    plt.colorbar()
                plt.grid(False)
                plt.title(f"Noisy Mean of Uncertainty Layer {l + 1}")
            plt.show()
            break
            
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(c_img_var[:, :, l], vmin=0, vmax=0.1, cmap="jet")
                plt.title(f"Denoised Variance of Uncertainty Layer {l + 1}")
                plt.colorbar()
                plt.grid(False)
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(n_img_var[:, :, l], vmin=0, vmax=0.1, cmap="jet")
                plt.title(f"Noisy Variance of Uncertainty Layer {l + 1}")
                plt.colorbar()
                plt.grid(False)
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(c_img_p_mean[:, :, l], vmax=1, vmin=0, cmap="jet")
                plt.colorbar()
                plt.grid(False)
            plt.title("Denoised Mean of Probability")
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(n_img_p_mean[:, :, l], vmin=0, vmax=1, cmap="jet")
                plt.grid(False)
                plt.colorbar()
            plt.title("Noisy Mean of Probability")
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(c_img_p_var[:, :, l], vmin=0, vmax=0.07, cmap="jet")
                plt.colorbar()
                plt.grid(False)
            plt.title("Denoised var of Probability")
            plt.show()
    
            plt.figure(figsize=(50, 5))
            for l in range(9):
                plt.subplot(1, 9, l + 1)
                plt.imshow(n_img_p_var[:, :, l], vmin=0, vmax=0.07, cmap="jet")
                plt.grid(False)
                plt.colorbar()
            plt.title("Noisy var of Probability")
            plt.show()
            break

In [ ]:
pcolor_plot(1)

In [ ]:
# interact(pcolor_plot, max_val=widgets.FloatSlider(min=1, max=10, step=0.5, value=10))

In [ ]:
n_preds.shape

In [ ]:
plt.figure(figsize=(50, 5))
for l in range(9):
    plt.subplot(1, 9, l + 1)
    plt.plot(range(n_preds_var.shape[0]), n_preds_var[:, :, l].var(axis=(1, 2)).cpu().detach().numpy(),
             marker='o', linestyle='-', color='orange')
    plt.plot(range(c_preds_var.shape[0]), c_preds_var[:, :, l].var(axis=(1, 2)).cpu().detach().numpy(),
             marker='o', linestyle='-', color='green')
    plt.title(f"Layer {l + 1}")
    plt.grid(True)
    plt.legend(["Noisy", "Denoised"])
plt.show()

In [ ]:
plt.figure(figsize=(50, 5))
for l in range(9):
    plt.subplot(1, 9, l + 1)
    plt.plot(range(n_preds_var.shape[0]), n_preds_mean[:, :, l].mean(axis=(1, 2)).cpu().detach().numpy(),
             marker='o', linestyle='-', color='orange')
    plt.plot(range(c_preds_var.shape[0]), c_preds_mean[:, :, l].mean(axis=(1, 2)).cpu().detach().numpy(),
             marker='o', linestyle='-', color='green')
    plt.title(f"Layer {l + 1}")
    plt.grid(True)
    plt.legend(["Noisy", "Denoised"])
plt.show()

In [ ]:
n_layer_uc_var.shape, c_layer_uc_var.shape

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(c_layer_uc_var.shape[0]),
         np.array([1.0 - torch.max(c_preds,dim=2).values.cpu().detach().numpy()]).squeeze().std(axis=(1, 2, 3)), marker='o', linestyle='-',
         color='green')

plt.plot(range(c_layer_uc_mean.shape[0]),
         np.array([1.0 - torch.max(n_preds,dim=2).values.cpu().detach().numpy()]).squeeze().std(axis=(1, 2, 3)), marker='o', linestyle='-',
         color='orange')

plt.grid(True)
plt.title("STD Uncertainty per Image")
plt.legend(["Denoised", "Noisy"])

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(n_layer_uc_var.shape[0]),
         np.array([1.0 - torch.max(c_preds,dim=2).values.cpu().detach().numpy()]).squeeze().mean(axis=(1, 2, 3)), marker='o',
         linestyle='-', color='green')

plt.plot(range(c_layer_uc_mean.shape[0]),
         np.array([1.0 - torch.max(n_preds,dim=2).values.cpu().detach().numpy()]).squeeze().mean(axis=(1, 2, 3)), marker='o',
         linestyle='-', color='orange')

plt.grid(True)
plt.title("Mean Uncertainty per Image")
plt.legend(["Denoised", "Noisy"])

plt.show()

plt.plot(range(50), np.max(c_preds[0, :, :, :, :], axis=1)[:, 100, 100], marker='o', color="orange")
plt.plot(range(50), np.max(n_preds[0, :, :, :, :], axis=1)[:, 100, 100], marker='o', color="g")
plt.title("Overall Probability")
plt.legend(["Denoised", "Noisy"])
plt.show()

plt.subplot(1, 2, 1)
plt.hist(np.max(c_preds[4, :, :, :, :], axis=1)[:, 100, 100], color="g")
plt.subplot(1, 2, 2)
plt.hist(np.max(n_preds[4, :, :, :, :], axis=1)[:, 100, 100], color="b")
plt.show()

np.mean(1.0 - np.max(c_preds, axis=2)[:, :, 100, 100], axis=1).shape

plt.figure(figsize=(12, 6))
plt.plot(range(40), np.mean(1.0 - np.max(c_preds, axis=2)[:, :, 100, 100], axis=1), marker="o")
plt.plot(range(40), np.mean(1.0 - np.max(n_preds, axis=2)[:, :, 100, 100], axis=1), marker="o")
plt.legend(["Denoised", "Noisy"])
plt.title("Mean at (100,100)")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(range(40), np.std(1.0 - np.max(c_preds, axis=2)[:, :, 100, 100], axis=1), marker="o")
plt.plot(range(40), np.std(1.0 - np.max(n_preds, axis=2)[:, :, 100, 100], axis=1), marker="o")
plt.legend(["Denoised", "Noisy"])
plt.title("Std at (100,100)")
plt.show()

n_preds[4, :, :, :, :][0, :, 100, 100]

np.max(n_preds[4, :, :, :, :], axis=1)[:, 100, 100]

n_preds[3, :, :, :, :][:, :, 100, 100]

c_preds[3, :, :, :, :][:, :, 100, 100]

for i, (c_img_mean, c_img_var, n_img_mean, n_img_var, c_img_p_mean, n_img_p_mean, c_img_p_var,
        n_img_p_var) in enumerate(
    zip(c_layer_uc_mean, c_layer_uc_var, n_layer_uc_mean, n_layer_uc_var, c_layer_prob_mean, n_layer_prob_mean,
        c_layer_prob_var, n_layer_prob_var)):
    print(c_img_mean)
    break

print(np.transpose(c_preds[0].mean(axis=0), (1, 2, 0))[65, 65, :])

In [ ]:
c_img, n_img = c_preds[4], n_preds[4]

In [ ]:
c_img_gt, n_img_gt = c_trues[4,0], n_trues[4,0]

In [ ]:
c_img.shape, n_img.shape, c_img_gt.shape, n_img_gt.shape

In [ ]:
for i in range(9):
    print('\t',torch.mean((1.0-torch.max(c_img,dim=1).values)*c_img_gt[i]),torch.std((1.0-torch.max(c_img,dim=1).values)*c_img_gt[i]),torch.mean((1.0-torch.max(n_img,dim=1).values)*n_img_gt[i]),torch.std((1.0-torch.max(n_img,dim=1).values)*n_img_gt[i]),"\n")

In [ ]:
layers=[]
for i in range(9):
    layers.append((np.mean((torch.mean((1.0-torch.max(c_img,dim=1).values))*c_img_gt[i]).cpu().detach().numpy()),np.std((torch.mean((1.0-torch.max(c_img,dim=1).values))*c_img_gt[i]).cpu().detach().numpy())))
pd.DataFrame(np.array(layers)).to_csv("clean.csv")
del layers

In [ ]:
layers=[]
for i in range(9):
    layers.append((np.mean((torch.mean((1.0-torch.max(n_img,dim=1).values))*n_img_gt[i]).cpu().detach().numpy()),np.std((torch.mean((1.0-torch.max(n_img,dim=1).values))*n_img_gt[i]).cpu().detach().numpy())))
pd.DataFrame(np.array(layers)).to_csv("noisy.csv")
del layers

In [ ]:
((1.0-torch.max(c_img,dim=1).values)*c_img_gt[0]).std()

In [ ]:
((1.0-torch.max(n_img,dim=1).values)*n_img_gt[0]).std()

In [ ]:
c_img.shape

In [ ]:
np.std(1.0-np.max(c_img.cpu().detach().numpy(),axis=1)),np.std(1.0-np.max(n_img.cpu().detach().numpy(),axis=1))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2

In [ ]:
img=cv2.imread(r"D:\Academics\IT\Python\Pytorch_ML\Shivlal_Sir\Self2SelfPlus_1\Cleaned_DataSet\test\NORMAL3.png")

In [ ]:
img=cv2.resize(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),(192,192))


In [ ]:
plt.imshow(img)
plt.grid(False)